In [1]:
# silence warning messages
import h2o
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/jupyter/.*')
h2o.init(nthreads = -1)

# Load the H2O library and start up the H2O cluter locally on your machine
import h2o

# Number of threads, nthreads = -1, means use all cores on your machine
# max_mem_size is the maximum memory (in GB) to allocate to H2O
h2o.init(nthreads = -1, max_mem_size = 8)



No instance found at ip and port: localhost:54321. Trying to start local jar...


JVM stdout: /var/folders/k_/kpp1czqs3957vq2pr5qngck00000gn/T/tmpvlbU_u/h2o_laurend_started_from_python.out
JVM stderr: /var/folders/k_/kpp1czqs3957vq2pr5qngck00000gn/T/tmpXvhOkJ/h2o_laurend_started_from_python.err
Using ice_root: /var/folders/k_/kpp1czqs3957vq2pr5qngck00000gn/T/tmpYASHvj


Java Version: java version "1.8.0_73"
Java(TM) SE Runtime Environment (build 1.8.0_73-b02)
Java HotSpot(TM) 64-Bit Server VM (build 25.73-b02, mixed mode)


Starting H2O JVM and connecting: .............. Connection successful!


H2O cluster uptime:,1 seconds 601 milliseconds
H2O cluster version:,3.8.3.2
H2O cluster name:,H2O_started_from_python_laurend_cqe986
H2O cluster total nodes:,1
H2O cluster total free memory:,3.56 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


H2O cluster uptime:,1 seconds 664 milliseconds
H2O cluster version:,3.8.3.2
H2O cluster name:,H2O_started_from_python_laurend_cqe986
H2O cluster total nodes:,1
H2O cluster total free memory:,3.56 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


In [2]:
#h2o.shutdown(prompt=False)
#h2o.init(nthreads = -1, max_mem_size = 8)

## Train a binary classification GBM model

In [3]:
# A small clean telecommunications sample dataset (https://www.ibm.com/communities/analytics/watson-analytics-blog/predictive-insights-in-the-telco-customer-churn-data-set/
telco_dataset = h2o.import_file("https://s3.amazonaws.com/h2o-smalldata/TelcoChurn.csv")
# select all columns as predictors except the customerID (which is like an index) and the response column
features_list = list(telco_dataset.columns[1:-1])
response_name = 'Churn'


Parse Progress: [##################################################] 100%


In [4]:
# Partition data into 70%, 15%, 15% chunks
# Setting a seed will guarantee reproducibility
splits = telco_dataset.split_frame(ratios=[0.75,0.15], seed=1234)

train = splits[0]
valid = splits[1]
test = splits[2]

In [5]:
# Import H2O GBM:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [6]:
# Initialize and train the GBM estimator:

gbm_fit1 = H2OGradientBoostingEstimator(model_id='gbm_fit1', seed=1234)
gbm_fit1.train(x=features_list, y=response_name, training_frame=train)


gbm Model Build Progress: [##################################################] 100%


In [7]:
pred = gbm_fit1.predict(valid)


gbm prediction Progress: [##################################################] 100%


In [8]:
performance_train = gbm_fit1.model_performance(train)
print performance_train.auc()
performance_valid = gbm_fit1.model_performance(valid)
print performance_valid.auc()

0.900653373457
0.853859853927


In [9]:
models_predictions = gbm_fit1.predict(valid)
print models_predictions 


gbm prediction Progress: [##################################################] 100%
predict           No        Yes
---------  ---------  ---------
Yes        0.49132    0.50868
No         0.824918   0.175082
Yes        0.0922927  0.907707
No         0.956509   0.0434913
No         0.750771   0.249229
Yes        0.276651   0.723349
No         0.921678   0.0783222
Yes        0.272807   0.727193
No         0.949678   0.0503224
Yes        0.316944   0.683056

[1029 rows x 3 columns]


In [10]:
intervention_cost = 3.0  # Cost of classification 
effectiveness = 0.1      # 10% of users will be influenced by this particular intervention

In [11]:
# this is temporary for debubgging only
# we can set a threshold to use for now (this will be a variable in the future)
newdata = valid
model = gbm_fit1
threshold = 0.6
pred = model.predict(newdata)
pred['predict'] = pred['Yes']>threshold


gbm prediction Progress: [##################################################] 100%


In [12]:
pred

predict,No,Yes
0,0.49132,0.50868
0,0.824918,0.175082
1,0.0922927,0.907707
0,0.956509,0.0434913
0,0.750771,0.249229
1,0.276651,0.723349
0,0.921678,0.0783222
1,0.272807,0.727193
0,0.949678,0.0503224
1,0.316944,0.683056


Now that we have a trained model and predictions (0/1) on a validation set, we can compute the confusion matrix to get the numbers of TP, FP, TN, FN.

In [13]:
conf = model.confusion_matrix()
print conf
print type(conf)


Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.338346064197: 


,No,Yes,Error,Rate
No,3238.0,644.0,0.1659,(644.0/3882.0)
Yes,272.0,1143.0,0.1922,(272.0/1415.0)
Total,3510.0,1787.0,0.1729,(916.0/5297.0)



<class 'h2o.model.confusion_matrix.ConfusionMatrix'>


In [14]:
conf_df = conf.table.as_data_frame()
print conf_df

              No     Yes   Error             Rate
0     No  3238.0   644.0  0.1659   (644.0/3882.0)
1    Yes   272.0  1143.0  0.1922   (272.0/1415.0)
2  Total  3510.0  1787.0  0.1729   (916.0/5297.0)


In [15]:
TN = conf_df.ix[0,1]  #True Negative
FN = conf_df.ix[0,2]  #False Negative
FP = conf_df.ix[1,1]  #False Positive
TP = conf_df.ix[1,2]  #True Positive

## Define cost parameters

- `unit_full_price`: Unit cost in USD, e.g. "cost of a Starbucks coffee"
- `unit_discount`: The discount of the coupon, e.g. 0.4 = 40% off
- `discount_effectiveness`: The effectiveness of the coupon to prevent churn, e.g. 0.3 = 30% of people who would have churned, but received the coupon, will decide to buy the unit at a discounted price instead of not buying (churning).

In [16]:
unit_full_price = 1.00
unit_discount = 0.4
discount_effectiveness = 0.3
discounted_unit_price = (1 - unit_discount) * unit_full_price
print discounted_unit_price

0.6


### Send coupon to people who are predicted to churn

For all the people who we predict as "Will Churn", we send them a coupon to try to get them not to churn.  There are two groups in this population:

1. TP: Apply the coupon and 30% of these folks will decide to buy the discounted item instead of not buying.  Company makes money.
2. FP: These people were not going to churn -- so now they have 40% off coupon for something they were going to pay full price for.  Company loses money.

In [17]:
# Total reward of TP group: TP * 0.6 * 0.3  #Discounted price is $0.60, which is 40% off of $1.00
TP_value = TP * discounted_unit_price * discount_effectiveness 
print TP_value

# Total cost of FP group: FP * 0.4 * 1.00  #Coupon is $0.40, which is 40% of $1.00, lost for each FP
FP_value = FP * unit_full_price * unit_discount * -1
print FP_value

205.74
-108.8


### Don't send coupon to people who are not predicted to churn

For all the people who we predict as "Will Not Churn", we don't bother to send them a coupon.  There are two groups in this population:

1. TN: These people were not going to churn, and they didn't get a coupon, so there's no loss or gain here.
2. FN: These people do churn, and since they didn't get an inticing coupon in the mail, there is nothing to prevent them from churning.  These are lost customers.  Company loses money.

In [18]:
# Total cost of FN group: 30% of these churns could have been saved at a 40% discount
FN_value = FN * discounted_unit_price * discount_effectiveness * -1
print FN_value

# Total cost/reward of TN group: Nothing, the intervention has no effect on the outcome for this population
TN_value = 0.00
print TN_value 

-115.92
0.0


In [19]:
intervention_net_value = TP_value + FP_value + TN_value + FN_value
print intervention_net_value

-18.98


### Intervention Value Function

Ok, lets wrap all this up in a function and try different interventions.

In [20]:
def intervention_value(intervention_params, newdata, model, threshold = None):
    ''' 
    intervention_params is a dict specifying intervention parameters
    model must be a binomial H2O model
    threshold is a number between 0 and 1
    newdata is an H2OFrame of test data
    '''
    
    # Parse parameters
    unit_full_price = intervention_params['unit_full_price']
    unit_discount = intervention_params['unit_discount']
    discount_effectiveness = intervention_params['discount_effectiveness']
    discounted_unit_price = (1 - unit_discount) * unit_full_price #CHANGED THIS FROM unit_cost
    
#     assert(unit_discount > 0.0)
    
    if threshold is not None:
        # Update the predictions using specified threshold
        pred = model.predict(newdata)
        pred['predict'] = pred['Yes']>threshold
    
    # Confusion matrix
    conf = model.confusion_matrix()
    print conf
    conf_df = conf.table.as_data_frame()
    TN = conf_df.ix[0,1]  #True Negative
    FN = conf_df.ix[0,2]  #False Negative
    FP = conf_df.ix[1,1]  #False Positive
    TP = conf_df.ix[1,2]  #True Positive
    
    # Total reward of TP group: TP * 0.6 * 0.3  #Discounted price is $0.60, which is 40% off of $1.00
    TP_value = TP * discounted_unit_price * discount_effectiveness 
    print TP_value

    # Total cost of FP group: FP * 0.4 * 1.00  #Coupon is $0.40, which is 40% of $1.00, lost for each FP
    FP_value = FP * unit_full_price * unit_discount * -1
    print FP_value
    
    # Total cost of FN group: 30% of these churns could have been saved at a 40% discount
    FN_value = TN * discounted_unit_price * discount_effectiveness * -1
    print FN_value

    # Total cost/reward of TN group: Nothing, the intervention has no effect on the outcome for this population
    TN_value = 0.00
    print TN_value 
    
    intervention_net_value = TP_value + FP_value + TN_value + FN_value
    print 'Value of intervention is %.2f'%intervention_net_value
    return intervention_net_value
    

In [21]:
# Let's try some interventions:

intervention1 = {'unit_full_price': 1.00,
                 'unit_discount': 0.4,
                 'discount_effectiveness': 0.3}

ival1 = intervention_value(intervention_params = intervention1, newdata = test, model = model, threshold = 0.2)


gbm prediction Progress: [##################################################] 100%

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.338346064197: 


,No,Yes,Error,Rate
No,3238.0,644.0,0.1659,(644.0/3882.0)
Yes,272.0,1143.0,0.1922,(272.0/1415.0)
Total,3510.0,1787.0,0.1729,(916.0/5297.0)



205.74
-108.8
-582.84
0.0
Value of intervention is -485.90


In [22]:
# A unit discount of $0.00 should produce a value of intervention of 0.00

intervention2 = {'unit_full_price': 1.00,
                 'unit_discount': 0.1,
                 'discount_effectiveness': 0.3}

ival2 = intervention_value(intervention_params = intervention2, newdata = test, model = model, threshold = 0.2)


gbm prediction Progress: [##################################################] 100%

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.338346064197: 


,No,Yes,Error,Rate
No,3238.0,644.0,0.1659,(644.0/3882.0)
Yes,272.0,1143.0,0.1922,(272.0/1415.0)
Total,3510.0,1787.0,0.1729,(916.0/5297.0)



308.61
-27.2
-874.26
0.0
Value of intervention is -592.85


In [23]:
# A unit discount of $0.00 should produce a value of intervention of 0.00... hmm maybe something is wrong above?

intervention2 = {'unit_full_price': 1.00,
                 'unit_discount': 0.0,
                 'discount_effectiveness': 0.3}

ival = intervention_value(intervention_params = intervention2, newdata = test, model = model, threshold = None)


Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.338346064197: 


,No,Yes,Error,Rate
No,3238.0,644.0,0.1659,(644.0/3882.0)
Yes,272.0,1143.0,0.1922,(272.0/1415.0)
Total,3510.0,1787.0,0.1729,(916.0/5297.0)



342.9
-0.0
-971.4
0.0
Value of intervention is -628.50
